> 运行前请安装dhg: `pip install git+https://github.com/iMoonLab/DeepHypergraph.git`

In [7]:
import sys
sys.path.append("../")  # 添加项目根目录到路径中

In [8]:
import torch
torch.manual_seed(199)
import torch.optim as optim
from torch import nn

import dhg

import hgp
from hgp.models import HGNNP,CHGNN,BDLiner
from hgp.loss import loss_bs_matrix,loss_bs_matrix_mega
from hgp.utils import from_pickle_to_hypergraph
from hgp.function import StraightThroughEstimator

DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
DEVICE

device(type='cuda', index=0)

In [9]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 3

h_hyper_prmts["convlayers1"] = {"in_channels": 1330, "out_channels": 1024, "use_bn": False, "drop_rate": 0.4}
h_hyper_prmts["convlayers3"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers4"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers5"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.3}

l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":128, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer2"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer3"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.1}
l_hyper_prmts["linerlayer4"] = {"in_channels":32, "out_channels":3, "use_bn":False, "drop_rate":0.1}

hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}
# fmt: on

#### 定义用于训练的类Trainer

In [10]:
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.   
    """
    # fmt: on
    def __init__(self, net, X, hg):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.layers = nn.ModuleList()
        for n in net:
            self.layers.append(n.to(DEVICE))   
        self.layers.append(nn.Softmax(dim=1))

    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch): 
        self.train()
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [11]:
from hgp.utils import from_pickle_to_adj
A = from_pickle_to_adj("../data/cora",unique=True)

[tensor([   0,    0,    0,  ..., 1328, 1329, 1329]),
 tensor([ 933,  936,  938,  ..., 1085, 1136, 1137])]

In [18]:
G = from_pickle_to_hypergraph("../data/cora")
edges, _ = G.e
G.num_e, G.num_v
G.H

tensor(indices=tensor([[   0,    0,    1,  ..., 1328, 1329, 1329],
                       [ 872,  875,  161,  ..., 1017, 1064, 1065]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(1330, 1413), nnz=4370, layout=torch.sparse_coo)

In [13]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
X = torch.eye(n=G.num_v)
embedding_net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
partition_net = BDLiner(hyper["l_hyper_prmts"]).to(DEVICE)

hgnn_trainer = Trainer(net=[embedding_net, partition_net], X=X, hg=G).to(DEVICE)

optim = optim.Adam(hgnn_trainer.parameters(), lr=4e-5, weight_decay=5e-8)
hgnn_trainer.optimizer = optim

In [14]:
# for name,p in hgnn_trainer.named_parameters():
#     print(name,p.shape)
# hgnn_trainer.layers
for n,p in hgnn_trainer.named_parameters():
    print(n,p)

layers.0.layers.0.theta.weight Parameter containing:
tensor([[-0.0138,  0.0145,  0.0074,  ..., -0.0130, -0.0147, -0.0027],
        [-0.0272, -0.0073,  0.0124,  ...,  0.0089,  0.0073,  0.0174],
        [ 0.0016, -0.0160,  0.0167,  ...,  0.0133, -0.0062,  0.0131],
        ...,
        [ 0.0060,  0.0022,  0.0170,  ...,  0.0260,  0.0265, -0.0233],
        [ 0.0164, -0.0179, -0.0143,  ..., -0.0068,  0.0012, -0.0168],
        [ 0.0262, -0.0263, -0.0137,  ...,  0.0023,  0.0186, -0.0157]],
       device='cuda:0', requires_grad=True)
layers.0.layers.0.theta.bias Parameter containing:
tensor([ 0.0102, -0.0036,  0.0125,  ...,  0.0084, -0.0239, -0.0192],
       device='cuda:0', requires_grad=True)
layers.0.layers.1.theta.weight Parameter containing:
tensor([[-0.0198,  0.0135,  0.0277,  ...,  0.0073, -0.0167,  0.0141],
        [-0.0149, -0.0045, -0.0054,  ..., -0.0087,  0.0194,  0.0164],
        [ 0.0073,  0.0309, -0.0059,  ...,  0.0087, -0.0113, -0.0213],
        ...,
        [ 0.0261,  0.0033, -0

In [15]:
# temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
# for epoch in range(4500):
#     loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
#     # train
#     temp_loss_total += loss
#     temp_loss1 += loss_1
#     temp_loss2 += loss_2
#     # validation
#     if epoch % 10 == 0:
#         print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
#         print(f"                , loss1: {temp_loss1.item() / 10}")
#         print(f"                , loss2: {temp_loss2.item() / 10}")
#         print(f"=================================")
#         sys.stdout.flush()
#         temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

In [16]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
best_loss = 1000
seed = 1
for i in range(100,200,1):
    torch.manual_seed(i)
    X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
    X = torch.eye(n=G.num_v)
    embedding_net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
    partition_net = BDLiner(hyper["l_hyper_prmts"]).to(DEVICE)
    hgnn_trainer = Trainer(net=[embedding_net, partition_net], X=X, hg=G).to(DEVICE)
    import torch.optim as optim
    optim = optim.Adam(hgnn_trainer.parameters(), lr=4e-5, weight_decay=5e-8)
    hgnn_trainer.optimizer = optim
    for epoch in range(4500):
        loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
        if loss_1 < best_loss:
            best_loss = loss_1
        # train
        temp_loss_total += loss
        temp_loss1 += loss_1
        temp_loss2 += loss_2
        # validation
        if epoch % 10 == 0:
            print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
            print(f"                , loss1: {temp_loss1.item() / 10}")
            print(f"                , loss2: {temp_loss2.item() / 10}")
            print(f"=================================")
            sys.stdout.flush()
            temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
print("=================================")
print("best loss:",best_loss)
print("seed:",i)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

260